NUMERIC COLUMNS:

max_salary

views

med_salary

min_salary

applies

original_listed_time

expiry

closed_time

listed_time

normalized_salary

In [2]:
#IMPORTS

import sqlite3
import pandas as pd
import numpy as np

In [3]:
#run this only after completing sqliteDBsetup.py execution
#DATABASE CONNECTION

file_path = r"C:\Users\Sarah\Code\ProyectoABD\database\linkedindatabase.db"
conn = sqlite3.connect(file_path)

Info:\
Salaries are provided either hourly, bi-weekly, monthly, or yearly. They also either contain a range (low-high), or a singular "median" value.

normalized_salary is a crude method to convert all of them to the same scale, by taking the average of low and high, and converting everything to an annual salary by multiplying by the number of weeks/months in a year

Note that they're still in different currencies

In [4]:
query = "SELECT job_id, max_salary, pay_period, views, med_salary, min_salary, applies, original_listed_time, expiry, closed_time, listed_time, currency, normalized_salary FROM postings"
df = pd.read_sql(query, conn)
df.head()
df.to_sql('numerical_postings', conn, if_exists='replace', index = False)
conn.commit()

In [14]:
df.loc[df['job_id'] == 3904911743]

,job_id,max_salary,pay_period,views,med_salary,min_salary,applies,original_listed_time,expiry,closed_time,listed_time,currency,normalized_salary
94881,3904911743,120000000.0,YEARLY,31.0,NaN,85000000.0,2.0,1713448770000,1729000576000,NaN,1713448770000,USD,102500000.0


In [6]:
df_numerical = pd.read_sql('SELECT * FROM numerical_postings', conn)
df_numerical.head()

,job_id,max_salary,pay_period,views,med_salary,min_salary,applies,original_listed_time,expiry,closed_time,listed_time,currency,normalized_salary
0,921716,20.0,HOURLY,20.0,NaN,17.0,2.0,1713397508000,1715989508000,NaN,1713397508000,USD,38480.0
1,1829192,50.0,HOURLY,1.0,NaN,30.0,NaN,1712857887000,1715449887000,NaN,1712857887000,USD,83200.0
2,10998357,65000.0,YEARLY,8.0,NaN,45000.0,NaN,1713277614000,1715869614000,NaN,1713277614000,USD,55000.0
3,23221523,175000.0,YEARLY,16.0,NaN,140000.0,NaN,1712895812000,1715487812000,NaN,1712895812000,USD,157500.0
4,35982263,80000.0,YEARLY,3.0,NaN,60000.0,NaN,1713451943000,1716043943000,NaN,1713451943000,USD,70000.0


In [7]:
print(df_numerical.loc[df_numerical['max_salary'].idxmax()])

job_id                     3904911743
max_salary                120000000.0
pay_period                     YEARLY
views                            31.0
med_salary                        NaN
min_salary                 85000000.0
applies                           2.0
original_listed_time    1713448770000
expiry                  1729000576000
closed_time                       NaN
listed_time             1713448770000
currency                          USD
normalized_salary         102500000.0
Name: 94881, dtype: object


In [8]:
df_numerical['currency'].value_counts(dropna=False)

currency
None    87776
USD     36058
EUR         6
CAD         3
BBD         2
AUD         2
GBP         2
Name: count, dtype: int64

In [9]:
#CONVERTING NULL TO USD
#df_numerical['currency'] = df_numerical['currency'].fillna('USD')

#CONVERTING CURRENCIES TO USD
conversion_rates = {'USD': 1, 'EUR': 1.18, 'GBP': 1.38, 'AUD': 0.74, 'CAD': 0.79, 'BBD': 0.50}

df_numerical['max_salary'] = df_numerical.apply(
    lambda x: conversion_rates[x['currency']] * x['max_salary']
    if not pd.isna(x['currency']) and not pd.isna(x['max_salary'])
    else np.nan,
    axis=1
)

df_numerical['max_salary'] = df_numerical.apply(
    lambda x: conversion_rates[x['currency']] * x['max_salary']
    if not pd.isna(x['currency']) and not pd.isna(x['max_salary'])
    else np.nan,
    axis=1
)

df_numerical['min_salary'] = df_numerical.apply(
    lambda x: conversion_rates[x['currency']] * x['min_salary']
    if not pd.isna(x['currency']) and not pd.isna(x['min_salary'])
    else np.nan,
    axis=1
)

df_numerical['med_salary'] = df_numerical.apply(
    lambda x: conversion_rates[x['currency']] * x['med_salary']
    if not pd.isna(x['currency']) and not pd.isna(x['med_salary'])
    else np.nan,
    axis=1
)

df_numerical['normalized_salary'] = df_numerical.apply(
    lambda x: conversion_rates[x['currency']] * x['normalized_salary']
    if not pd.isna(x['currency']) and not pd.isna(x['normalized_salary'])
    else np.nan,
    axis=1
)

df_numerical['currency'].value_counts(dropna=False)

currency
None    87776
USD     36058
EUR         6
CAD         3
BBD         2
AUD         2
GBP         2
Name: count, dtype: int64

In [ ]:
#CONVERTING PAY PERIOD TO ANNUAL SALARY
time_multiplier = {'HOURLY': 40*52, 'WEEKLY': 52, 'BIWEEKLY': 26, 'MONTHLY': 12, 'YEARLY': 1}

def convert_to_annual_max(row):
    if not pd.isna(row['max_salary']):
        if not pd.isna(row['pay_period']):
            if not (row['pay_period'] == 'HOURLY' and row['max_salary'] > 10000):
                return time_multiplier[row['pay_period']] * row['max_salary']
    return row['max_salary']
    
def convert_to_annual_med(row):
    if not pd.isna(row['med_salary']):
        if not pd.isna(row['pay_period']):
            if not (row['pay_period'] == 'HOURLY' and row['med_salary'] > 10000):
                return time_multiplier[row['pay_period']] * row['med_salary']
    return row['med_salary']

def convert_to_annual_min(row):
    if not pd.isna(row['min_salary']):
        if not pd.isna(row['pay_period']):
            if not (row['pay_period'] == 'HOURLY' and row['min_salary'] > 10000):
                return time_multiplier[row['pay_period']] * row['min_salary']
    return row['min_salary']
    



df_numerical['max_salary'] = df_numerical.apply(
    convert_to_annual_max,
    axis=1
)


df_numerical['min_salary'] = df_numerical.apply(
    convert_to_annual_min,
    axis=1
)

df_numerical['med_salary'] = df_numerical.apply(
    convert_to_annual_med,
    axis=1
)

df_numerical['pay_period'].value_counts(dropna=False)

pay_period
None        87776
YEARLY      20628
HOURLY      14741
MONTHLY       518
WEEKLY        177
BIWEEKLY        9
Name: count, dtype: int64

In [12]:
df_numerical['max_salary'].value_counts(dropna=False)

max_salary
NaN         94056
150000.0      648
100000.0      634
120000.0      610
90000.0       541
            ...  
58968.0         1
333400.0        1
178600.0        1
89303.0         1
230300.0        1
Name: count, Length: 5334, dtype: int64

In [13]:
print(df_numerical.loc[df_numerical['max_salary'].idxmax()])

job_id                     3904911743
max_salary                120000000.0
pay_period                     YEARLY
views                            31.0
med_salary                        NaN
min_salary                 85000000.0
applies                           2.0
original_listed_time    1713448770000
expiry                  1729000576000
closed_time                       NaN
listed_time             1713448770000
currency                          USD
normalized_salary         102500000.0
Name: 94881, dtype: object


Now all salary data is in usd, annual pay

In [15]:
df_numerical.to_sql('clean_numerical_postings', conn, if_exists='replace', index = False)
conn.commit()

In [38]:
conn.close()